# The North Remembers Project

In [1]:
%load_ext autoreload
%autoreload 2

### 取舍

数据存储用 github, 或者 bitbucket


提供导出为 rss, epub, pdf, github 自带检查历史的功能

不追求实效性



外链图片不作为重点
外链图片和视频链接, url 链接, 一并修复, 换行字符串等也一并处理, 七牛云?
md 中的 Latex 和 代码检测也在这里修复, pygment

### 调用关系





#### Collector

处理用户发出的命令

可以是 zhihu answer url, 问题页, 收藏夹页, 需要带有过滤条件
用户可能在过滤条件后再做修补删减

需要转为具体的爬虫任务, 并创建文件夹
单页收集到特定的文件夹, 否则收集到一个自动命名的文件夹

在文件夹下生成task

Resources
global_config
task_config (scanlist)


Collector.append(url, options)



#### Watcher



任务调度, 监视文件夹, 根据 task.yaml 抓取页面

task.yaml 也记录历史

task_type: single_page, 
update_batch: 10


track_new: true


通过扫描已有的待抓取任务, 在手动触发, 达到特定时间后, 分派抓取任务

    Watcher
        .watch()
        .remember()
        .status()
        .load_config_yaml
        .save_config_yaml
        .add_task add_tasks



Task

被调用抓取单一页面内容, 或者抓取一个 url 列表
抓取单一页面内容将存为 md, 不存 html

    Task
        ZhihuAnswerPageTask
        ZhihuAnswerIndexTask
        ZhihuColumnPageTask
        ZhihuColumnIndexTask

Task 会使用 Page.save / load

    .last_page()
    .fetch()
    .save() 传递一些参数给 page 如 watch_date
    .to_yaml_text()
    
    .priority
    .is_page_type
    .should_fetch
    .content_is_changed



ZhihuColumnPage
ZhihuColumnLister


Fetcher.request(url) => dict()? Page()

Page.write to / load from disk

PageTask.run => Fetcher.request(url) => New Page
PageTask.update => New Page.write
IndexTask.run => Fetcher.request(url) => [New Tasks]


Watcher -> PageTask -> Page (tmpl, fetcherAPI)
                            ZhihuColumnPage
                            ZhihuAnswerPage
                            WeixinArticlePage


Watcher -> IndexTask -> append more tasks
                        ZhihuColumnIndex
    





#### Fetcher

Page

页面内容分为 元数据 + 文章主体 + 评论, 评论先放放


Page.load()  返回一个读盘的页面

以md文件为单位

    ZhihuColumnPage
        .load(path)
        .write(path)
        .compare()

    attr
        task = ForeignKeyField(Task, related_name='task')
        watch_date = DateTimeField()
        title = CharField(null=True)
        author = CharField(null=True)
        content = TextField(null=True)
        metadata = TextField(null=True)
        topic = TextField(null=True)
        question = TextField(null=True)
        comment = TextField(null=True)





#### Cleaner

处理, 清理文本的规则



Exporter

根据 Template 转换为 rss, epub, pdf, 以及 github diff 预览

图片转存也算在 Template 里

In [ ]:
描述

以更加简洁清晰的排版阅读知乎. (这个需求有简悦)

用户指定一个知乎网站上的问题, zhihumark 抓取相应的回答内容, 以 markdown 文档的形式提供给用户阅读和下载. (提供 md 下载没用)





功能

可以自由定制想抓取的内容, 这包括: 抓取指定答主/收藏夹名/问题/话题下的回答, 抓取赞同数量超过N的回答.
可以设置黑名单, 不抓取特定的问题或者答主.
抓取评论, 以会话的形式组织起来, 让上下文相关的评论出现在一起, 尽量滤除没有价值的评论.
将收集到的一系列文档保存为epub或者pdf, 或者批量下载 markdown 文档, 以便离线阅读.
制作成 feed 以便放入 rss 阅读器, 获取实时更新.


观念

我们也不显示其他人搜索了什么关键字, 获取了什么页面, 下载了哪些电子书, 看到什么取决于你

我们相信评论是回答(或文章)可以分割的一部分, 许多评论很水, 有些很有价值.

相信评论区是对回答的重要补充, 也有随回答保留的必要, 但需处理评论区良莠不齐, 引用回复混乱的问题.



页面设计

单页面网站应用, 基本上类似搜索引擎的风格和排版.

输入框 - 接受输入ID或URL 答主/收藏夹名/问题名/话题名

选项 - 保留最前的n个回答?
选项 - 赞同数至少是?
选项 - 保留评论? 评论对话中至少有多少 likes
屏蔽 - 不感兴趣的话题, 问题, 答主
返回 - markdown文件列表

批量下载按钮

In [ ]:
def test_fetch_history():
  # 美国是不是正在为瓦解中国做准备？ - 张俊麒的回答
  # url = 'https://www.zhihu.com/question/40622033/answer/88139460'
  
  # 有哪些令人拍案叫绝的临场反应？ - 大盗贼霍老爷的回答
  url = 'https://www.zhihu.com/question/29125576/answer/54187388'
  
  # 机器人教育这种不考试、以娱乐为主的教育对于中小学生及幼儿的意义何在？ - 冷哲的回答"
  url = 'https://www.zhihu.com/question/26999365/answer/93421419'
  
  
  # 2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 小马的回答"
  url = 'https://www.zhihu.com/question/40910547/answer/123021503'
  
  # Smart Beta 投资方法 - 陈颖的专栏 量化哥"
  url = 'https://zhuanlan.zhihu.com/p/21936302'
  
  
  # 美国南北战争的真正原因是什么？ - talich的回答"
  url = 'https://www.zhihu.com/question/19679278/answer/15122151'
  # query = (Page.select(Page, Task)
  #          .join(Task)
  #          .where((Task.page_type == 'zhihu_article') & (Page.title.contains('最前沿')))
  #          .group_by(Page.task)
  #          .having(Page.watch_date == fn.MAX(Page.watch_date))
  #          .limit(9999))
  t = Task.select().where(Task.url == url).get()
  
  
  contents = []
  for page in t.pages:

    print(page.version)
    print(page.title)
    print(page.topic)
    contents.append(page.content)
#     print(page.content)
  return contents
contents = test_fetch_history()

In [ ]:
def split_text(text):
  col = []
  for sent in re.findall('.*?[！|，|。|？|\n|；|~|～|：|\)|\]]', text):
    col.append(sent)
  
  return col

print(split_text(a))

In [ ]:
print(len(contents))
# c1, c2, c3, c4, *_, c7 = contents
from difflib import unified_diff
def compare_text(t1, t2, prefix=''):
  changes = [l for l in unified_diff(t1, t2)]

  return changes


import re
for i, (a, b) in enumerate(zip(contents[:-1], contents[1:])):
#   if i == 0:
#     continue
  if (a!=b):
    print('a!=b on', i, i+1)
    
    pat = re.compile('\S')
    for d in unified_diff(split_text(a), split_text(b), n=2, lineterm=''):
      print(d)
  
  print('\n\n\n\n\n\n')
#   if i >= 5:
#     break
  
# compare_text(c1, c2)
# compare_text(c2, c3)

In [ ]:
'iesytwe'.split('?')

In [ ]:

# from unidiff import PatchSet
# diff = urllib2.urlopen('https://github.com/matiasb/python-unidiff/pull/3.diff')
# encoding = diff.headers.getparam('charset')
# patch = PatchSet(diff, encoding=encoding)


### 原 models test exec




In [ ]:

'''
####### #######  ###### #######
   ##   ##      ##         ##
   ##   ######   #####     ##
   ##   ##           ##    ##
   ##   ####### ######     ##
'''

def test_new_task():
  # url = 'https://www.zhihu.com/question/30595784/answer/49194862'
  # 如何看待许知远在青年领袖颁奖典礼上愤怒「砸场」？
  url = 'https://www.zhihu.com/question/22316395/answer/100909780'
  url = 'https://www.zhihu.com/question/47220155/answer/118154455'
  url = 'https://www.zhihu.com/question/49962599/answer/118716273'
  url = 'https://zhuanlan.zhihu.com/p/19837940'
  url = 'https://zhuanlan.zhihu.com/p/20639779'
  url = 'https://zhuanlan.zhihu.com/p/20153329'
  url = 'https://zhuanlan.zhihu.com/p/21281864'
  url = 'https://zhuanlan.zhihu.com/p/19964142'
  task = Task.add(url=url)
  print(task)
  # task.watch()


def test_readd_task():
  url = 'http://www.zhihu.com/question/22513722/answer/21967185' # 火车票涨价
  task = Task.add(url=url)
  task = Task.add(url=url)
  url = 'https://www.zhihu.com/question/30957313/answer/50266448' # 古典音乐
  task = Task.add(url=url)
  url = 'https://www.zhihu.com/question/40056948/answer/110794550' # 四万亿
  task = Task.add(url=url)

def test_seed_add_tasks():
  urls = '''
    https://www.zhihu.com/question/40305228/answer/86179116
    https://www.zhihu.com/question/36466762/answer/85475145
    https://www.zhihu.com/question/33246348/answer/86919689
    https://www.zhihu.com/question/39906815/answer/88534869

    https://www.zhihu.com/question/40700155/answer/89002644
    https://www.zhihu.com/question/36380091/answer/84690117
    https://www.zhihu.com/question/33246348/answer/86919689
    https://www.zhihu.com/question/35254746/answer/90252213
    https://www.zhihu.com/question/23618517/answer/89823915

    https://www.zhihu.com/question/40677000/answer/87886574

    https://www.zhihu.com/question/41373242/answer/91417985
    https://www.zhihu.com/question/47275087/answer/106335325
    https://www.zhihu.com/question/47275087/answer/106335325 买不起房是房价太高还是工资太低？
    https://www.zhihu.com/question/36129534/answer/91921682  印度经济会在本世纪追上中国吗？
    https://www.zhihu.com/question/22513722/answer/21967185  火车票涨价是否能解决春运问题？
    https://www.zhihu.com/question/27820755/answer/107267228 裸辞后怎样解释以获工作机会？
  '''
  for url in datalines(urls):
    url = url.split(' ')[0]
    task = Task.add(url=url)
    task | puts()



def test_one_watch():
  task = Task.select().order_by(-Task.id).get()
  task | puts()
  task.watch()

def test_another_watch():
  url = 'http://www.zhihu.com/question/22513722/answer/21967185' # 火车票涨价
  task = Task.select().where(Task.url == url).get()
  task | puts()
  task.watch()


def test_hot_answer():
  url = 'http://www.zhihu.com/question/39288165/answer/110207560'
  url = 'https://www.zhihu.com/question/50763374/answer/122822226'
  url = 'https://www.zhihu.com/question/40910547/answer/123021503'
  url = 'https://zhuanlan.zhihu.com/p/21478575'
  url = 'https://www.zhihu.com/question/40103788/answer/124499334'
  task = Task.add(url=url)
  task.watch()


  task.last_page.to_local_file(folder='test', fetch_images=False)



def test_watch_all():
  Task.multiple_watch(sleep_seconds=10, limit=4)


def test_report():
  Task.report()



def test_to_local_file():
  # page = Page.select().order_by(-Page.id).get()

  # page = Page.select(Page.topic).distinct().where(Page.topic.contains('房')).limit(5)
  # q = Page.select(Page.id).distinct()
  # for p in q:
  #   print(p)
  query = (Page.select(Page, Task)
           .join(Task)
           .where(Page.author == 'chenqin')  # .where(Page.topic.contains('建筑'))
           .group_by(Page.task)
           .having(Page.watch_date == fn.MAX(Page.watch_date))
           .limit(8800))
  for page in query:
    log(page.title)
    # log(page.metadata)
    # page.to_local_file(folder='chen', fetch_images=False)
# test_to_local_file()

def test_tools():
  import pylon
  pylon.generate_figlet('task', fonts=['space_op'])
  pylon.generate_figlet('page', fonts=['space_op'])
  pylon.generate_figlet('test', fonts=['space_op'])


def test_load_json():
  import json

  print(json.loads(open('mockup_topic_answers.json', encoding='utf-8').read()))


def test_banned_modes():
  url = 'https://www.zhihu.com/question/40679967/answer/88310495'
  # 政府推出开放小区政策的真正目的是什么？ 2201 孟德尔 回答建议修改：政治敏感
  pass


def test_query_task():
  s = 'https://www.zhihu.com/question/48737226/answer/113036453'
  s = 'https://www.zhihu.com/question/47220155/answer/118154455'
  s = 'https://www.zhihu.com/question/49545583/answer/116529877'

  t = Task.select().where(Task.url == s)
  t = t.get()
  log(t.title)
  log(t.id)






def test_explore():
  # Tweet.select(fn.COUNT(Tweet.id)).where(Tweet.user == User.id)
  query = (Task
           .select(Task, fn.COUNT(Page.id).alias('fetched_count'))
           .join(Page)
           .group_by(Task.title)
           .limit(50)
           .offset(200)
           .order_by(fn.COUNT(Page.id).desc()))

  for task in query:
    log(task.title + ' : ' + str(task.fetched_count) + '  task_id ' + str(task.id))




def test_explore_watching_results_diff():

  s = '''
  # 美国是不是正在为瓦解中国做准备？ - 张俊麒的回答 : 16  task_id 46
  # 为什么很少看到患者砍莆田系医生的报道？ - 玄不救非氪不改命的回答 : 3  task_id 196
  # 为什么很难证伪马克思主义理论？ - 玄不救非氪不改命的回答 : 3  task_id 360
  # 为什么快速浏览一段内容的时候，很容易看到自己感兴趣的部分？ - 采铜的回答 : 3  task_id 742
  # 为什么拿广州恒大淘宝队与中国国家男子足球队做对比？ - 玄不救非氪不改命的回答 : 3  task_id 492
  # 如何看待里约奥运陈欣怡药检呈阳性反应？ - 玄不救非氪不改命的回答 : 3  task_id 2393
  # 为什么厌恶「国粉」的知乎用户远多于厌恶「毛粉」的？ - chenqin的 : 3  task_id 3313
  # 2016 年中国的经济状况很差吗？真实状况是怎样的？ - 垒起善城堡的积木 : 3 task_id 2387
  # 如何看待2016年7月人民币贷款增幅里9.8成为房贷？ - 匿名用户的回答 : 3 task_id 2386




  # 怎样评价「游戏不汉化就差评」的行为？ - cOMMANDO的回答 : 9  task_id 4471
  # 既然有报道说人类的基因片段只占DNA序列总长的不到10%，那么这几个问题怎么解答？ - Mandelbrot的回答 : 9  task_id 676
  # 智商低的人真的不适合玩需要动脑子的游戏么？ - 匿名用户的回答 : 9  task_id 3461
  # 暴雪，Valve，拳头，谁更厉害？ - cOMMANDO的回答 : 9  task_id 2597
  # 有一个稀有的姓是一种怎样的体验？ - 冷哲的回答 : 9  task_id 1041
  # 有什么影视作品是当时演员名气不大，现在看来是全明星阵容出演？ - 玄不救非氪不改命的回答 : 9  task_id 484
  # 有关白龙尾岛的历史，哪些是有据可查的？ - 书生的回答 : 9  task_id 712
  # 有哪些「智商税」？ - 谢熊猫君的回答 : 9  task_id 2406
  # 有哪些令人拍案叫绝的临场反应？ - 大盗贼霍老爷的回答 : 9  task_id 4107
  # 有哪些可怕的故事？ - 大盗贼霍老爷的回答 : 9  task_id 4137
  # 有哪些长得比较逆天的动物？ - Mandelbrot的回答 : 9  task_id 648
  # 有文化有多可怕？ - 寺主人的回答 : 9  task_id 5787
  # 机器人教育这种不考试、以娱乐为主的教育对于中小学生及幼儿的意义何在？ - 冷哲的回答 : 9  task_id 1015
  # 毛花三年打败蒋然后走三十年弯路的目的，都是为后三十年的改革开放走资、大国崛起做铺垫扫平道路的么？ - 书生的回答 : 9  task_id 99
  # 水旱蝗汤中的汤指的到底是谁？ - 书生的回答 : 9  task_id 104
  # 河南的地理位置那么好，为什么经济落后？ - 大盗贼霍老爷的回答 : 9  task_id 4105
  # 為什麼蒋介石被称为运输大队长？求详? - 书生的回答 : 9  task_id 701
  # 玩《狼人杀》时你有什么屡试不爽的秘技诀窍？ - 汪诩文的回答 : 9  task_id 3526

  现在网络上很多人黑一些伟人，比如说周半期，黑鲁迅。他们是什么心态？ - 书生的回答 : 9  task_id 97
  看美剧、英剧学英语有什么有效的方法吗？ - 采铜的回答 : 9  task_id 787
  章鱼的智商到底有多高，为什么有人说它们的智商可以统治世界? - Mandelbrot的回答 : 9  task_id 588
  类似 AlphaGo 的人工智能在游戏王、万智牌等卡牌游戏中胜率如何？ - 莫名的回答 : 9  task_id 3724
  给 59 分强行不给过的老师是一种怎么样的存在？ - chenqin的回答 : 9  task_id 3317
  网络上有哪些广为流传的「历史真相」其实是谣言？ - 马前卒的回答 : 9  task_id 5089
  美国南北战争的真正原因是什么？ - talich的回答 : 9  task_id 2571
  美国发动伊拉克战争的核心原因到底是什么？ - 冷哲的回答 : 9  task_id 1332
  美国最高法院大法官 Scalia 的去世将会带来怎样的影响？ - talich的回答 : 9  task_id 4423
  美国有人在开车在路上故意把川普的竞选宣传牌碾倒，如何评价这种因为不同政见而破坏对方财物的行为？ - talich的回答 : 9  task_id 4412
  装逼成功是怎样一种体验？ - 大盗贼霍老爷的回答 : 9  task_id 4048
  谁最应该被印在人民币上面？ - 蜂鸟的回答 : 9  task_id 2274
  豆瓣的核心用户都有什么特点？ - 十一点半的回答 : 9  task_id 84
  赌场有哪些看似不起眼，实则心机颇深的设计？ - 第一喵的回答 : 9  task_id 3158
  赌场有哪些看似不起眼，实则心机颇深的设计？ - 肥肥猫的回答 : 9  task_id 3151
  雷锋是个什么样的人，怎么客观评价雷锋？ - 书生的回答 : 9  task_id 96
  鲁迅和秋瑾的关系好吗？ - 书生的回答 : 9  task_id 91
  1927 年蒋介石为什么要清党？ - 冷哲的回答 : 8  task_id 1415
  1949年以后的中国本土设计的建筑中，哪些能称得上是有思想的好建筑？ - Chilly的回答 : 8  task_id 3104
  2015 年初，中国制造业形势有多严峻？ - 稻可道 稻子的稻的回答 : 8  task_id 2239
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - Bee Mad的回答 : 8  task_id 2104
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 君临的回答 : 8  task_id 2205
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 小马的回答 : 8  task_id 5752
  ISIS 是一个什么样的组织？它的资金是哪来的？ - 罗晓川的回答 : 8  task_id 39
  Lambda 表达式有何用处？如何使用？ - 涛吴的回答 : 8  task_id 2008
  Signal Weighting---基于因子IC的因子权重优化模型 - 陈颖的专栏 量化哥 : 8  task_id 6007
  Smart Beta 投资方法 - 陈颖的专栏 量化哥 : 8  task_id 6017
  ofo 获滴滴数千万美元C轮投资，然后呢？ - 曲凯的专栏 创投方法论 : 8  task_id 5656
  《文明 6 》中的背景音乐都有什么来历？ - PenguinKing的回答 : 8  task_id 6250
  《权力的游戏》你觉得最可怜的人是谁？ - 苏鲁的回答 : 8  task_id 4576
  《蒋介石日记》和《毛泽东选集》差距有多大？ - 马前卒的回答 : 8  task_id 5355
  「心灵鸡汤」式的文章错在哪？ - 赵皓阳的回答 : 8  task_id 584
  '''

  for line in datalines(s):
    task_id = int(line.split('task_id')[-1][1:])
    # log(task_id)
    task = Task.select().where(Task.id == task_id).get()
    log(task)
    # log(task.pages)
    # log(task.last_page)
    contents = [fix_in_compare(p.content) for p in task.pages]
    # questions = [fix_in_compare(p.question) for p in task.pages]
    titles = [p.title for p in task.pages]

    # metas = [p.metadata for p in task.pages]
    # for meta in metas:
    #   log(meta)
    # compare_text_sequence(titles, label='titles')
    # compare_text_sequence(questions, label='questions')
    compare_text_sequence(contents, label='contents')

    log('\n\n\n')



def fix_in_compare(text):
  import re
  img_reg = r'\n*\!\[\]\((https?://pic[^()]+?(\.jpg|\.png|\.gif))\)\n*'
  pattern_img_start_inline = re.compile(img_reg)
  def replace_img_start_inline(mat):
    # 保证生成的 *.md 图片在新的一行
    s = mat.group(0)
    while not s.startswith('\n\n'):
      s = '\n' + s
    while not s.endswith('\n\n'):
      s = s + '\n'
    return s

  text = pattern_img_start_inline.sub(replace_img_start_inline, text)

  pattern_img_https = re.compile(r'http://pic(\d)\.zhimg\.com')
  text = pattern_img_https.sub(r'https://pic\1.zhimg.com', text)
  return text


def compare_text(t1, t2, prefix=''):
  import difflib
  changes = [l for l in difflib.ndiff(t1.split('\n'), t2.split('\n')) if l.startswith(('+ ', '- '))]
  for change in changes:
    log(prefix + change)
  return changes


def compare_text_sequence(texts, label=''):
  from pylon import dedupe
  from pylon import windows

  texts = list(dedupe(texts))
  if len(texts) > 1:
    log('detect changed {}'.format(label))
    for t1, t2 in windows(texts, length=2, overlap=1):
      compare_text(t1, t2, prefix='  ')
  else:
    log('nothing changed {}'.format(label))



def test_explore_voteup_thanks():
  '''感谢赞同比跟文章质量没啥关系'''
  query = (Page.select(Page, Task)
           .join(Task)
           .where((Task.page_type == 'zhihu_answer'))
           .group_by(Page.task)
           .having(Page.watch_date == fn.MAX(Page.watch_date))
           .limit(50)
           .order_by(fn.Random())
           )

  def thanks_voteup_ratio(page):
    thanks = int(page.metadata.split('thanks: ')[1].split(' ')[0])
    voteup = int(page.metadata.split('voteup: ')[1].split(' ')[0])
    return round(thanks / voteup, 3)

  # for page in query:
  #   log(page.title)

  pages = sorted(query, key=thanks_voteup_ratio)
  for page in pages:
    log(page.title)
    log(repr(page.content[:500]))
    log(thanks_voteup_ratio(page))
    log('-----------------\n\n\n')


### Test Collector

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from collector import Collector


col = Collector(project_path='D:\DataStore\The North Remembers - Art')

# topic_id = 19644231 # 古建筑
# topic_id = 19582176 # 建筑设计
# topic_id = 19573393 # 建筑史
# topic_id = 19568972 # 建筑学

In [ ]:
url = 'https://zhuanlan.zhihu.com/c_186387023'
tip = '何为现实？拉普拉斯之妖与薛定谔猫之决战'
# col.add(url, tip, lister_option=None, page_option=None)
col.add(url)
str(col)

In [15]:
urls = [
  'https://www.zhihu.com/topic/19644231/top-answers', # 古建筑
  'https://www.zhihu.com/topic/19582176/top-answers', # 建筑设计
  'https://www.zhihu.com/topic/19573393/top-answers', # 建筑史
  'https://www.zhihu.com/topic/19568972/top-answers', # 建筑学
]

col.add_multiple(urls, lister_option={'zhihu_min_voteup': 5000})
str(col)

generate_tip get 古建筑
generate_tip get 建筑设计
generate_tip get 建筑史
generate_tip get 建筑学
generate_tip get 古建筑
generate_tip get 建筑设计
generate_tip get 建筑史
generate_tip get 建筑学
Watcher folder D:\DataStore\The North Remembers - Art/知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学 created


'<Collector #1187701854336 path=D:\\DataStore\\The North Remembers - Art>'

In [ ]:
# collector.add(weixin_page_url)
# collector.add(zhihu_column_url)
# collector.add(zhihu_anwser_id=xxx)
# collector.add(zhihu_author_id=xxx)
# collector.add(column_id=xxx)

# collector.save(file=xxx)
# ...
# collector.rescan()
# collector.save(file=xxx)
# collector = Collector.load(file=xxx.yaml)

### Test Watcher

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from watcher import Watcher
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\旗舰评论——战略航空军元帅的旗舰')
#w = Watcher(r'D:\Coding\TheNorthRemembers\test\知乎 - 温酒的回答')
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\经济')
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\人文主义谈话录')
# w = Watcher(r'D:\DataStore\The North Remembers - Science\混乱博物馆')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\篝火旁的故事')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\沉思的托克维尔')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\何为现实？拉普拉斯之妖与薛定谔猫之决战')

w = Watcher(r'D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学')


loaded config_data option lister: 
{'enabled': True,
 'limit': 200,
 'max_cycle': '15day',
 'min_cycle': '1hour',
 'weight': 0.5,
 'zhihu_min_voteup': 5000}
loaded config_data option page: 
{'enabled': True, 'max_cycle': '1day', 'min_cycle': '1hour', 'weight': 0.5}
loaded config_data lister tasks 4
loaded config_data page tasks 80
Watcher.add_tasks result status , Counter({'added': 4})
Watcher.add_tasks result status , Counter({'added': 80})


In [3]:
t = w.tasks
t[0]
t[0].option


OrderedDict([('enabled', True),
             ('max_cycle', '15day'),
             ('min_cycle', '1hour'),
             ('weight', 0.5),
             ('limit', 200),
             ('zhihu_min_voteup', 5000),
             ('folder',
              'D:\\DataStore\\The North Remembers - Art\\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学'),
             ('url', 'https://www.zhihu.com/topic/19644231/top-answers'),
             ('tip', '知乎话题 - 古建筑'),
             ('timestamp',
              OrderedDict([('task_add', '2019-07-31 14:01:27'),
                           ('last_watch', '2019-07-31 14:02:17'),
                           ('last_change', '2019-07-31 14:02:17'),
                           ('next_watch', '2019-07-31 15:02:17')])),
             ('version', 1)])

In [11]:
w.watch()

Watcher.watch should fetch 0 lister_type tasks

Watcher.watch should fetch 80 page_type tasks

Task.run page request: <Task #1739067346840> https://www.zhihu.com/question/29598382/answer/69829208 (ver. 0)
    你们见过最丑的家装是怎样的？ - 匿名用户的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/你们见过最丑的家装是怎样的？ - 匿名用户的回答.md done
Watcher.watch page task done: <Task #1739067346840>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739067346280> https://www.zhihu.com/question/271634898/answer/364995861 (ver. 0)
    有哪些对你很有冲击力的设计？ - 匿名用户的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/有哪些对你很有冲击力的设计？ - 匿名用户的回答.md done
Watcher.watch page task done

Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/AutoCAD 制图中的各种比例问题如何解决？ - 猪小宝的回答.md done
Watcher.watch page task done: <Task #1739008277080>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739008277360> https://www.zhihu.com/question/22827881/answer/27280438 (ver. 0)
    家居中有哪些设计不合理的地方？ - 鸡汤泡面的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/家居中有哪些设计不合理的地方？ - 鸡汤泡面的回答.md done
Watcher.watch page task done: <Task #1739008277360>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739008277976> https://www.zhihu.com/question/19754430/answer/66364271 (ver. 0)
    如何处理「极简设计理念」与「设计无内涵」之间的矛盾？ - eno艾诺的回答
    task add: 21 days ago, la

Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/为什么越来越多的人不愿意去设计院了？ - 外贸南瓜RAY的回答.md done
Watcher.watch page task done: <Task #1739008679160>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739008678488> https://www.zhihu.com/question/264213676/answer/281806004 (ver. 0)
    迄今为止你见过最惊艳的建筑是哪个？ - 若瑜的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/迄今为止你见过最惊艳的建筑是哪个？ - 若瑜的回答.md done
Watcher.watch page task done: <Task #1739008678488>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739008678824> https://www.zhihu.com/question/318969984/answer/678899664 (ver. 0)
    我应不应该向父母要个iPad ？ - 千书的回答
    task add: 21 days ago, last watch: 

Task.run page request: <Task #1739062425080> https://www.zhihu.com/question/27701927/answer/121824448 (ver. 0)
    宜家有哪些性价比高的收纳神器？ - 吉大将的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/宜家有哪些性价比高的收纳神器？ - 吉大将的回答.md done
Watcher.watch page task done: <Task #1739062425080>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/.task.yaml" done
Task.run page request: <Task #1739068122280> https://www.zhihu.com/question/25528331/answer/31038897 (ver. 0)
    长城这么矮究竟能拦住什么？ - 刘鑫的回答
    task add: 21 days ago, last watch: None, last change None, next watch: 21 days ago 
Task.run fetch page done
write D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学/长城这么矮究竟能拦住什么？ - 刘鑫的回答.md done
Watcher.watch page task done: <Task #1739068122280>


done, save_config_yaml "D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建

ZhihuParseError: 

In [ ]:
w.tasks[2].should_fetch

### Test Generate Feed 

In [65]:
from recorder import generate_feed

generate_feed(r'D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学')

<Page #1739095927888> AutoCAD 制图中的各种比例问题如何解决？ - 猪小宝的回答 (ver. 1, 2019-08-21 16:08:29) 
<Page #1739062474064> 中国传统建筑中的宫、殿、堂、楼、阁、轩、榭、居、斋、馆、亭等各有什么特点？ - 大卫独舞的回答 (ver. 1, 2019-07-31 14:06:24) 
<Page #1739062472944> 中国农村建筑为什么越来越没有美感？ - jian mi的回答 (ver. 1, 2019-08-21 16:16:53) 
<Page #1739062527368> 中国历史上为什么很少有像欧洲石制教堂宫殿这样高大宏伟的建筑？ - 荙铭爱糖糖的回答 (ver. 1, 2019-07-31 14:09:02) 
<Page #1739062499256> 中国古建筑的斗拱结构是否符合力学原理？ - 猪小宝的回答 (ver. 1, 2019-07-31 14:07:37) 
<Page #1739062498416> 中国有哪些现存的宏伟至极的古建筑？ - 旧集散地的回答 (ver. 1, 2019-08-21 16:17:28) 
<Page #1739095835536> 中国有哪些现存的宏伟至极的古建筑？ - 櫼栌落的回答 (ver. 1, 2019-08-21 16:18:57) 
<Page #1739095837944> 为什么 9·11 恐怖袭击中双子塔在一段时间的燃烧后倒塌了？ - 猪小宝的回答 (ver. 1, 2019-08-21 16:11:10) 
<Page #1739064452040> 为什么国内机场、火车站都修建得那么高？ - 胖墩的回答 (ver. 1, 2019-08-21 16:14:21) 
<Page #1739064451144> 为什么现在的建筑不盖成古代那样了？ - 晓渔的回答 (ver. 1, 2019-08-21 16:10:12) 


<BaseResponse 13427 bytes [200 OK]>

In [66]:
resp = _

In [67]:
print(bytes.decode(resp.response[0]))

<?xml version="1.0" encoding="utf-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <title type="text">知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学</title>
  <id>feed_url--request.url</id>
  <updated>2019-08-21T16:18:57+08:00</updated>
  <link href="url--request.url_root" />
  <link href="feed_url--request.url" rel="self" />
  <generator>Werkzeug</generator>
  <entry xml:base="feed_url--request.url">
    <title type="text">中国传统建筑中的宫、殿、堂、楼、阁、轩、榭、居、斋、馆、亭等各有什么特点？ - 大卫独舞的回答</title>
    <id>https://www.zhihu.com/question/32237946/answer/734344973</id>
    <updated>2019-07-31T14:06:24+08:00</updated>
    <published>2019-07-31T14:06:24+08:00</published>
    <link href="https://www.zhihu.com/question/32237946/answer/734344973" />
    <author>
      <name>article.get('author_name')</name>
    </author>
    <content type="html">
---

title:       中国传统建筑中的宫、殿、堂、楼、阁、轩、榭、居、斋、馆、亭等各有什么特点？ - 大卫独舞的回答
url:         https://www.zhihu.com/question/32237946/answer/734344973
author:      大卫独舞 (davidwhut)
voteup:      15166 赞同

In [47]:
resp.response

[b'<?xml version="1.0" encoding="utf-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <title type="text">\xe7\x9f\xa5\xe4\xb9\x8e\xe8\xaf\x9d\xe9\xa2\x98 - \xe5\x8f\xa4\xe5\xbb\xba\xe7\xad\x91, \xe5\xbb\xba\xe7\xad\x91\xe8\xae\xbe\xe8\xae\xa1, \xe5\xbb\xba\xe7\xad\x91\xe5\x8f\xb2, \xe5\xbb\xba\xe7\xad\x91\xe5\xad\xa6</title>\n  <id>feed_url--request.url</id>\n  <updated>2019-08-21T16:22:03+08:00</updated>\n  <link href="url--request.url_root" />\n  <link href="feed_url--request.url" rel="self" />\n  <generator>Werkzeug</generator>\n  <entry xml:base="feed_url--request.url">\n    <title type="text">\xe4\xb8\xad\xe5\x9b\xbd\xe4\xbc\xa0\xe7\xbb\x9f\xe5\xbb\xba\xe7\xad\x91\xe4\xb8\xad\xe7\x9a\x84\xe5\xae\xab\xe3\x80\x81\xe6\xae\xbf\xe3\x80\x81\xe5\xa0\x82\xe3\x80\x81\xe6\xa5\xbc\xe3\x80\x81\xe9\x98\x81\xe3\x80\x81\xe8\xbd\xa9\xe3\x80\x81\xe6\xa6\xad\xe3\x80\x81\xe5\xb1\x85\xe3\x80\x81\xe6\x96\x8b\xe3\x80\x81\xe9\xa6\x86\xe3\x80\x81\xe4\xba\xad\xe7\xad\x89\xe5\x90\x84\xe6\x9c\x89\xe4\xbb

### Fetch comments MaxRetryError

### generate token

In [ ]:
from tools import generate_zhihu_token
generate_zhihu_token()

In [ ]:
from fetcher_api import zhihu

In [ ]:
client = zhihu.client

In [ ]:
c = client.collection(133030984)
int("133030984")

In [ ]:
for ans in c.answers:
  print(ans.author.name)

In [ ]:
from zhihu_oauth import ZhihuClient

TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)


In [ ]:
page = client.from_url('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
from fetcher_api.zhihu import fetch_zhihu_article

In [ ]:
fetch_zhihu_article('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
import os
os.path.basename('D:\Coding\TheNorthRemembers\page.py')

In [ ]:
os.path.__file__

In [ ]:
import wechatsogou

In [ ]:
ws_api = wechatsogou.WechatSogouAPI()

In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
list(ws_api.search_article('马前卒工作室'))

In [ ]:
topic_id = 11
f'generate_tip {topic_id}'